In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

class ReaidyIoSwish(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        sig = torch.sigmoid(x)
        ctx.save_for_backward(x, sig)
        return x * sig

    @staticmethod
    def backward(ctx, grad_output):
        x, sig = ctx.saved_tensors
        grad = grad_output * (sig + x * sig * (1 - sig))
        return grad


def reaidyioswish(x):
    return ReaidyIoSwish.apply(x)


x = torch.randn(4, 4, dtype=torch.double, requires_grad=True)

def AutogradEnginereaidytest_function(input):
    return reaidyioswish(input)

print(
    "My Gradient check passed it's correct:",
    torch.autograd.gradcheck(AutogradEnginereaidytest_function, (x,), eps=1e-6, atol=1e-4)
)

x_train = torch.randn(200, 1)
y_train = 3 * x_train + 0.5 + 0.2 * torch.randn(200, 1)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(1, 32)   
        self.fc2 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = reaidyioswish(x)
        x = self.fc2(x)
        return x
model = NeuralNetwork()
loss_fn = nn.MSELoss()
opt = optim.Adam(model.parameters(), lr=0.005)  
for epoch in range(40):
    opt.zero_grad()
    out = model(x_train)
    loss = loss_fn(out, y_train)
    loss.backward()
    opt.step()

    if (epoch + 1) % 10 == 0:
        print("Epoch", epoch + 1, "Loss:", round(loss.item(), 4))


My Gradient check passed it's correct: True
Epoch 10 Loss: 6.2698
Epoch 20 Loss: 3.8131
Epoch 30 Loss: 1.835
Epoch 40 Loss: 0.5738
